# Edge Trees Analysis

## Import settings

In [ ]:
import json
from pathlib import Path
import os

#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO.json"
#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEVPLI.json"
#settings_file = r"C:\SVK_utveckling\settings_SEVPLI.json"
settings_file = r"C:\Users\SE1K4H\Desktop\SVK-Analys-Filer\settings_test_PostGIS.json" # Working on Elsas's computer

try:
    if not os.path.exists(settings_file):
        raise FileNotFoundError(f"Could not find {settings_file}")

    with open(settings_file, 'r', encoding='utf-8') as file:
        settings = json.load(file)

    # TODO: check which ones are needed and not
    run_ID = settings["run_ID"]
    powerline_list = settings["powerline_list"]
    local_dir = settings["local_folder"]
    results_gdb_template = settings["results_gdb_template"]
    domains_folder = settings["domains_folder"]
    scandate_file = settings["scandate_file"]
    cvd_LEDNINGSGATA_path = os.path.join(domains_folder, "cvd_LEDNINGSGATA.txt")
    powerlines_folder = settings["powerlines_folder"]
    LG_polygons = settings["LG_polygons"]
    station_polygons = settings["station_polygons"]
    order_gdb = settings["order"]
    ogr2ogr_path = settings["ogr2ogr_path"]   
    proj_lib_path = settings["proj_lib_path"]
    gdal_data_path = settings["gdal_data_path"]
    DEFAULT_DB_NAME = settings["default_db_name"]
    DB_NAME = settings["db_name"]
    USER = settings["db_user"]
    PASSWORD = settings["db_password"]
    HOST = settings["host"]
    PORT = settings["port"] 

    if None in (run_ID, powerline_list, local_dir, results_gdb_template, domains_folder, scandate_file, powerlines_folder, LG_polygons, station_polygons, order_gdb, ogr2ogr_path, proj_lib_path, gdal_data_path, DEFAULT_DB_NAME, DB_NAME, USER, PASSWORD, HOST, PORT):
        raise KeyError(f"One or more keys are missing in {settings_file}")
    
    results_gdb = os.path.join(local_dir, f"results_{run_ID}_PostGIS.gdb")
    
    print(f"Settings loaded successfully.")
except json.JSONDecodeError as e:
    print(f"Error: Invalid JSON format in {settings_file}")
except KeyError as e:
    print(f"Error: {e}")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: Unexpected error: {e}")

## Load PostGIS methods

In [30]:
import psycopg

# TODO can move these kind of methods to utils or similiar?
def connect_to_database(db_name):
    try:
        conn = psycopg.connect(dbname=db_name, user=USER, password=PASSWORD, host=HOST, port=PORT)
        conn.autocommit = True
        return conn
    except psycopg.Error as e:
        print(f"Error connecting to {db_name}: {e}")
        return None
    
def execute_query(conn, query, data=None, fetch=False):
    try:
        with conn.cursor() as cur:
            #print(f"Running query: {query.as_string(conn)}") # Add this line for debugging purpose
            cur.execute(query, data or ())
            if fetch:
                return cur.fetchall()
    except psycopg.Error as e:
        print(f"Error executing query: {e}")

## Copy template gdb

In [ ]:
import os
import shutil

if os.path.exists(results_gdb):
    print(f"{results_gdb} already exists")
else: 
    shutil.copytree(results_gdb_template, results_gdb)

## Merge SKB text files for all blocks of a powerline

In [ ]:
from pathlib import Path
import glob
import fileinput
import pandas as pd
import os

# TODO add exception handling.

def combine_blocks(row):
    LG = row["LG"]
    line = row["line"]
    line_dir = Path(powerlines_folder) / LG / f"line_{line}"
    
    block_dir = os.path.join(line_dir, "kantträd", "block")
    combined_blocks_path = os.path.join(line_dir, "kantträd", "SKB_raw.txt")
    
    # Merge files for all blocks into one
    merge_blocks(block_dir, "*.txt", combined_blocks_path)
    print(f"Successfully merged all edge trees blocks for {LG}/line_{line} into text file {combined_blocks_path}.")

def merge_blocks(src_dir, search_pattern, dst_file):
    blocks = glob.glob(os.path.join(src_dir, search_pattern))
    with open(dst_file, "w") as fh:
        input_lines = fileinput.input(blocks)
        fh.writelines(input_lines)

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(combine_blocks, axis=1)
print(f"Done with all power lines")

## Add Edge Trees to PostGIS Tables 

In [ ]:
import psycopg
from pathlib import Path
import pandas as pd
import os
from psycopg import sql #TODO update to and install psycopg, the latest version.

cur = None
conn = None

def create_SKB_XYmZ_table(row):
    LG = row["LG"]
    line = row["line"]
    table_name = f"{LG.lower()}_{line}_skb_xymz"
    line_dir = Path(powerlines_folder) / LG / f"line_{line}"
    SKB_dir = line_dir / "kantträd"
    SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")
    drop_table_if_exists_query = sql.SQL("DROP TABLE IF EXISTS {table_name}").format(table_name=sql.Identifier(table_name))
    create_query = sql.SQL("CREATE TABLE {table_name}(objectid SERIAL PRIMARY KEY, x DOUBLE PRECISION, y DOUBLE PRECISION, z FLOAT, dz FLOAT, mz FLOAT, shape geometry(POINTZ, 3006))").format(table_name=sql.Identifier(table_name)) #TODO potentially change to 5845

    try:
        # Step 1: Create table for powerline
        execute_query(conn_db, drop_table_if_exists_query)
        execute_query(conn_db, create_query)
        
        # Step 2: Insert edge trees into powerline table 
        with open(SKB_file_in, 'r') as src_file:
            for file_line in src_file:
                l_split = file_line.split(' ')
                x = float(l_split[0])
                y = float(l_split[1])
                z = float(l_split[2])
                dz = float(l_split[3])
                mz = z - dz
                
                #TODO potentially change to 5845, also check if SRID is needed on table level as well
                # TODO check if I should change to ST_SetSRID(ST_MakePoint(x, y, z), 3006)
                insert_query = sql.SQL("""
                    INSERT INTO {table_name} (x, y, z, dz, mz, shape)
                    VALUES (%s, %s, %s, %s, %s, ST_GeomFromText(%s, 3006))
                """).format(table_name=sql.Identifier(table_name))

                pointz_string = f"POINTZ({x} {y} {mz})"
                tree_data = (x, y, z, dz, mz, pointz_string)
                execute_query(conn_db, insert_query, tree_data)

    except psycopg.Error as e:
        print("Error while working with PostgreSQL:", e)   

try: 
    conn_db = connect_to_database(DB_NAME)
    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)

    if conn_db:
        powerlines_df.apply(create_SKB_XYmZ_table, axis=1)
    
    print("Successfully inserted edge trees to PostGIS database.") 

finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

## Conduct distance calculations

In [ ]:
import psycopg
from pathlib import Path
import pandas as pd

def calculate_distances(row, conn_db):
    #TODO se över SQL query om de kan förbättras, optimeras
    #TODO Borde ta bort columner först om de existerar 
    LG = row["LG"]
    line = row["line"]
    table_name = f"{LG.lower()}_{line}_skb_xymz"
    wire_table_name = f"{LG.lower()}_{line}_fas"
    
    add_columns_query = sql.SQL("""ALTER TABLE {table_name}
        ADD COLUMN IF NOT EXISTS avst_mz_fas DOUBLE PRECISION,
        ADD COLUMN IF NOT EXISTS avst_hori DOUBLE PRECISION, 
        ADD COLUMN IF NOT EXISTS avst_fas DOUBLE PRECISION;""").format(table_name=sql.Identifier(table_name))
    
    avst_mz_fas_query = sql.SQL("""UPDATE {table_name} p
        SET avst_mz_fas = subquery.distance
        FROM (
            SELECT DISTINCT ON (p.objectid)  
                p.objectid,
                ST_3DDistance(p.shape, l.shape) AS distance
            FROM 
                {table_name} p, 
                {wire_table_name} l
            WHERE 
                ST_3DDWithin(p.shape, l.shape, 100)
            ORDER BY 
                p.objectid, distance ASC
        ) AS subquery
        WHERE p.objectid = subquery.objectid;""").format(table_name=sql.Identifier(table_name), wire_table_name=sql.Identifier(wire_table_name))

    avst_fas_query = sql.SQL("""UPDATE {table_name} p
        SET avst_fas = subquery.distance
        FROM (
            SELECT objectid, avst_mz_fas - dz AS distance
            FROM {table_name}
        ) AS subquery
        WHERE p.objectid = subquery.objectid;""").format(table_name=sql.Identifier(table_name))

    avst_hori_query = sql.SQL("""UPDATE {table_name} p
        SET avst_hori = subquery.distance
        FROM (
            SELECT DISTINCT ON (p.objectid)  
                p.objectid,
                ST_Distance(p.shape, l.shape) AS distance
            FROM 
                {table_name} p, 
                {wire_table_name} l
            WHERE 
                ST_DWithin(p.shape, l.shape, 100)
            ORDER BY 
                p.objectid, distance ASC
        ) AS subquery
        WHERE p.objectid = subquery.objectid;""").format(table_name=sql.Identifier(table_name), wire_table_name=sql.Identifier(wire_table_name))

    # Step 1: Add attributes
    execute_query(conn_db, add_columns_query)
    print(f"Columns were added successfully to table {table_name}")

    # Step 2: Calculate shortest distance between mz and phase.
    execute_query(conn_db, avst_mz_fas_query)
    print(f"avst_mz_fas was calculated succesfully for {LG}_{line}")

    # Step 3: Calculate the shortest distance between tree top (z) and phase with event of potential fall.
    execute_query(conn_db, avst_fas_query)
    print(f"avst_fas was calculated succesfully for {LG}_{line}")

    # Step 4: Calculate the horizontal distance between tree and phase.
    execute_query(conn_db, avst_hori_query)
    print(f"avst_hori was calculated succesfully for {LG}_{line}") 

try: 
    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
    conn_db = connect_to_database(DB_NAME)

    if conn_db:
        powerlines_df.apply(lambda row: calculate_distances(row, conn_db), axis=1)
    
    print("Successfully calculated distances for all wires.") #TODO printed on error, change this
except psycopg.Error as e:
    print("Error while working with PostgreSQL:", e) 
except Exception as e:
    print("Error: ", e)
finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

# Add Littera, Ursprung, Ursprung_Datum, Registreringsdatum

In [ ]:
import psycopg
import pandas as pd
from psycopg import sql
from datetime import date

def add_data(row):
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    regdatum = skanningsdatum[f"{LG}_{line}.0"]   
    table_name = f"{LG.lower()}_{line}_skb_xymz"

    #TODO ledningsgata, insamlingsmetod, matosakerhet x2 should be updated to domains, using foreign keys 
    add_columns_query = sql.SQL("""ALTER TABLE {table_name}
                                    ADD COLUMN IF NOT EXISTS rgdtm DATE, 
                                    ADD COLUMN IF NOT EXISTS ledningsgata FLOAT,
                                    ADD COLUMN IF NOT EXISTS insamlingsmetod FLOAT,
                                    ADD COLUMN IF NOT EXISTS matosakerhet_plan FLOAT,
                                    ADD COLUMN IF NOT EXISTS matosakerhet_hojd FLOAT,
                                    ADD COLUMN IF NOT EXISTS littera VARCHAR(255),
                                    ADD COLUMN IF NOT EXISTS ursprung VARCHAR(255),
                                    ADD COLUMN IF NOT EXISTS ursprung_datum DATE;""").format(table_name=sql.Identifier(table_name))

    # TODO Update littera to work for domain
    # TODO Investigate if % or placeholder should be used
    update_query = sql.SQL("""UPDATE {table_name}
                                SET rgdtm = {regdatum}, 
                                littera = {littera},
                                ursprung = {ursprung},
                                ursprung_datum = {lev_dat};""").format(table_name=sql.Identifier(table_name), 
                                                                       regdatum=sql.Placeholder("regdatum"), 
                                                                       littera=sql.Placeholder("littera"), 
                                                                       ursprung=sql.Placeholder("ursprung"), 
                                                                       lev_dat=sql.Placeholder("lev_dat"))
    updated_values = {"regdatum": regdatum, "littera": littera, "ursprung": urspr, "lev_dat": lev_dat}

    try:
        # Step 1: Add columns
        execute_query(conn_db, add_columns_query)
        print(f"Columns were added successfully to table {table_name}")
        
        # Step 2: Update Littera, Ursprung, Ursprung_Datum, Registreringsdatum 
        execute_query(conn_db, update_query, updated_values)
        print(f"Values were updated successfully in table {table_name}")

    except psycopg.Error as e:
        print("Error while working with PostgreSQL:", e)   

try: 
    conn_db = connect_to_database(DB_NAME)
    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
    df_skanningsdatum = pd.read_csv(scandate_file, sep="\t", header=0) #TODO english variables?
    skanningsdatum = {f"{row['LG']}_{row['line']}": row['skanningsdatum'] for _, row in df_skanningsdatum.iterrows()}
    urspr = "SWECO"
    lev_dat = str(date.today())

    if conn_db:
        powerlines_df.apply(add_data, axis=1) #TODO don't have to pass conn_db on the other places, do like this instead
    
    print("Successfully added columns and updated values.") 
except Exception as e:
    print("Error: ", e)
finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

## Merge edge trees for all powerlines into one table

In [ ]:
def list_skb_xymz_tables():
    select_tables_query = sql.SQL("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_name LIKE '%%skb_xymz' AND table_schema = 'public';
    """)

    result = execute_query(conn_db, select_tables_query, fetch=True)
    table_names = [t[0] for t in result]

    return table_names

def merge_tables(table_names):
    table_name = 'kantträd_oklippta'

    drop_table_if_exists_query = sql.SQL("DROP TABLE IF EXISTS {table_name}").format(table_name=sql.Identifier(table_name)) #TODO maybe these two queries can be combined into one?

    create_table_query = sql.SQL("""
        CREATE TABLE {table_name} AS
        TABLE {first_table} WITH NO DATA;
    """).format(table_name=sql.Identifier(table_name), first_table=sql.Identifier(table_names[0]))

    union_parts = []
    for table in table_names:
        part = sql.SQL("""
            SELECT * FROM (
                SELECT *
                FROM {table}
                ORDER BY objectid
            )
        """).format(table=sql.Identifier(table))
        union_parts.append(part)

    union_query = sql.SQL(" UNION ALL ").join(union_parts)

    insert_query = sql.SQL("""
        INSERT INTO {table_name} (objectid, x, y, z, dz, mz, shape, avst_mz_fas, avst_hori, avst_fas, rgdtm, ledningsgata, insamlingsmetod, matosakerhet_plan, matosakerhet_hojd, littera, ursprung, ursprung_datum)
        SELECT
            ROW_NUMBER() OVER () AS objectid, x, y, z, dz, mz, shape, avst_mz_fas, avst_hori, avst_fas, rgdtm, ledningsgata, insamlingsmetod, matosakerhet_plan, matosakerhet_hojd, littera, ursprung, ursprung_datum
        FROM (
            {unioned}
        )
    """).format(table_name=sql.Identifier(table_name), unioned=union_query)

    execute_query(conn_db, drop_table_if_exists_query)
    execute_query(conn_db, create_table_query)
    execute_query(conn_db, insert_query)

try: 
    conn_db = connect_to_database(DB_NAME)

    if conn_db:
        table_names = list_skb_xymz_tables()        
        if table_names:
            table_names = sorted(table_names)
            print(f"Found tables: {table_names}")
            merge_tables(table_names)
            print("Successfully merged all edge trees tables into one table.") #TODO This should not be printed if sql error occurs..
        else:
            print("No tables ending with 'skb_xymz' found.")
except Exception as e:
    print("Error: ", e)
finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

# Clip Intersection Points

In [ ]:
import os
import subprocess
import pandas as pd
from psycopg import sql

#TODO in general in all cells, take a look att exception handling
PG_CONNECTION = f"PG:host={HOST} dbname={DB_NAME} user={USER} password={PASSWORD} port={PORT}"

def convert_polygon_fc_to_postGIS(table_name):
    #TODO no vertical system was added, look more into that.
    ogr_command = [
        ogr2ogr_path,
        "-f", "PostgreSQL",
        PG_CONNECTION,
        order_gdb,
        "-a_srs", "EPSG:3006", # TODO check if RH2000 should be specified?
        "-nln", table_name,
        "-nlt", "MULTIPOLYGONZ", #TODO Check if this was the correct choice
        "-dim", "3",
        "-overwrite",    
        table_name
    ]

    try:
        os.environ["PROJ_LIB"] = proj_lib_path #TODO potentially solve this in another way. Now it has to be done in order to find -a_srs EPSG:3006
        os.environ["GDAL_DATA"] = gdal_data_path
        subprocess.run(ogr_command, check=True, capture_output=True, text=True)
        print(f"Successfully converted {order_gdb} to PostGIS as '{table_name}'.") #TODO verify this
    except subprocess.CalledProcessError as e:
        print(f"Error: Could not convert {order_gdb} to PostGIS. {e}")

def copy_table(new_table, copied_table):
    drop_table_if_exists_query = sql.SQL("DROP TABLE IF EXISTS {new_table}").format(new_table=sql.Identifier(new_table)) #TODO maybe these two queries can be combined into one?

    copy_table_query = sql.SQL("""
        CREATE TABLE {new_table} AS
        SELECT *
        FROM {copied_table};
    """).format(new_table=sql.Identifier(new_table), copied_table=sql.Identifier(copied_table))

    execute_query(conn_db, drop_table_if_exists_query)
    execute_query(conn_db, copy_table_query)
  
def extract_trees(table_name, where_clause):
    drop_table_if_exists_query = sql.SQL("DROP TABLE IF EXISTS {table_name}").format(table_name=sql.Identifier(table_name))

    avst_hori_query = sql.SQL("""
        CREATE TABLE {table} AS

        SELECT oklippta.*
        FROM kantträd_oklippta oklippta
        JOIN gng_ledningsgata lg
        ON ST_Intersects(oklippta.shape, lg.shape)
        WHERE {where_clause}

        UNION 

        SELECT oklippta.*
        FROM kantträd_oklippta oklippta
        JOIN gng_stationsomrade station
        ON ST_Intersects(oklippta.shape, station.shape)
        WHERE {where_clause}
    """).format(table=sql.Identifier(table_name), where_clause=sql.SQL(where_clause)) # TODO structure where_clause in another way?

    execute_query(conn_db, drop_table_if_exists_query)
    execute_query(conn_db, avst_hori_query)

def clip_trees(table_1, table_2):
    delete_from_query = sql.SQL("""
        DELETE FROM kantträd
        WHERE EXISTS (
            SELECT 1
            FROM {table_1} AS t1
            WHERE t1.objectid = kantträd.objectid
        ) 

        OR EXISTS (
            SELECT 1
            FROM {table_2} AS t2
            WHERE t2.objectid = kantträd.objectid
        );
    """).format(table_1=sql.Identifier(table_1), table_2=sql.Identifier(table_2))

    execute_query(conn_db, delete_from_query)

try:   
    table_1 = "tmp_traffpunkter_1"
    table_2 = "tmp_traffpunkter_2"

    convert_polygon_fc_to_postGIS('gng_ledningsgata')
    convert_polygon_fc_to_postGIS('gng_stationsomrade')
    conn_db = connect_to_database(DB_NAME)

    if conn_db:
        copy_table("kantträd", "kantträd_oklippta")
        # Extract trees within the powerline corridor and the station area that are within 7 meters from phase.
        extract_trees(table_1, "avst_hori < 7")
        # Extract trees within the powerline corridor and the station area with tree height <= 4.5 meters and distance to phase >= 5 meters.
        extract_trees(table_2, "dz <= 4.5 AND avst_fas >= 5")
        clip_trees(table_1, table_2)

except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

## Convert edge tree PostGIS table to file geodatabase

In [ ]:
# TODO Rename SKB_XYmZ feature class, dataset and table from the start to SKB_XYZ
# TODO Delete fields like x, y, z
# TODO check if shape should be mz or z, currently mz? 

import os 
import pandas as pd
import subprocess

def postGIS_to_gdb():
    table_name = "kantträd"

    ogr_command = [
        ogr2ogr_path,
        "-f", "OpenFileGDB",
        results_gdb,      
        pg_connection,
        "-a_srs", "EPSG:3006",
        "-nlt", "POINTZ",
        "-dim", "3",
        "-overwrite",
        "-sql", f"SELECT * FROM public.{table_name}",
        "-nln", table_name,
    ]

    try:
        subprocess.run(ogr_command, check=True, capture_output=True, text=True)
        print(f"Successfully converted PostGIS {table_name} into gdb {results_gdb}.")
    except subprocess.CalledProcessError as e:
        print(f"Error during import: {e}")

try:
    pg_connection = f"PG:host={HOST} dbname={DB_NAME} user={USER} password={PASSWORD} port={PORT}"
    ogr2ogr_path = ogr2ogr_path
    os.environ["PROJ_LIB"] = proj_lib_path #TODO potentially solve this in another way, but needed to specify EPSG:3006
    os.environ["GDAL_DATA"] = gdal_data_path

    postGIS_to_gdb()
except Exception as e:
    print("Error: ", e)

In [ ]:
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

#r"C:\SVK_2024\python_work_dir\results_250218.gdb"
#results_gdb = os.path.join(local_dir, f"results_{run_ID}.gdb")
#print(Path(results_gdb))

# arcpy.env.workspace = working_gdb
print(arcpy.env.workspace)
kanttrad = os.path.join(results_gdb, 'kantträd')
print(kanttrad)

arcpy.management.AssignDomainToField(kanttrad, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
arcpy.management.AssignDomainToField(kanttrad, "LITTERA", "cvd_LITTERA_LEDNING")
arcpy.management.AssignDomainToField(kanttrad, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
arcpy.management.AssignDomainToField(kanttrad, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
arcpy.management.AssignDomainToField(kanttrad, "Insamlingsmetod", "cvd_INMATNINGSMETOD")



#print(kanttrad)
#arcpy.env.workspace = kanttrad
#arcpy.env.workspace(kanttrad)
#print(arcpy.env.workspace)
#feature_classes = arcpy.ListFeatureClasses(feature_dataset="SKB_XYZ")
print("körning av cell klar")


# Kontroll
### Här kan man lägga in egna skript för kontroll av resultatet

In [ ]:
# FLytta till RBX-notebook.
# Listar alla LG - littera som finns i RBX-resultatet
# Jämför detta med logg-filen för att se att rätt ledningar
# är med i resultatet (inte alla ledningar har RBX-punkter)

import arcpy
import os
import geopandas as gpd

gdb_path = r"C:\SVK_2024\pythonkörningar\results_2501xx.gdb"
feature_class = "RBX_closest_points"

gdf = gpd.read_file(gdb_path, layer=feature_class)

gdf = gdf[["LEDNINGSGATA", "LITTERA"]]
unique_df = gdf.drop_duplicates().sort_values(["LEDNINGSGATA", "LITTERA"])

print(unique_df)